In [2]:
from chainer import Function, gradient_check, Variable, optimizers, serializers, utils, Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
import numpy as np


In [6]:
from sklearn import datasets
iris = datasets.load_iris()

In [202]:
X = iris.data.astype(np.float32)
Y = iris.target
N = Y.size
Y2 = np.zeros(3*N).reshape(N,3).astype(np.float32)
ans_name = []

for i in range(N):
    Y2[i,Y[i]] = 1.0
    k = np.argmax(Y2[i])
    ans_name.append(iris.target_names[k])
    
for i in range(0,N,20):
    print(i, iris.data[i], ans_name[i])

0 [5.1 3.5 1.4 0.2] setosa
20 [5.4 3.4 1.7 0.2] setosa
40 [5.  3.5 1.3 0.3] setosa
60 [5.  2.  3.5 1. ] versicolor
80 [5.5 2.4 3.8 1.1] versicolor
100 [6.3 3.3 6.  2.5] virginica
120 [6.9 3.2 5.7 2.3] virginica
140 [6.7 3.1 5.6 2.4] virginica


In [203]:
index = np.arange(N)
xtrain = X[index[index %2 !=0], :]
ytrain = Y2[index[index %2 !=0], :]

xtest = X[index[index %2 !=0], :] 
yans = Y[index[index %2 !=0]]

In [204]:
class IrisChain(Chain):
    def __init__(self, hidden_u):
        super(IrisChain,self).__init__(
        L1 = L.Linear(4,hidden_u),
        L2 = L.Linear(hidden_u,3),
        )
    def fwd(self,x):
        h1 = F.relu(self.L1(x))
        h2 = self.L2(h1)
        return h2
    def __call__(self,x,y):
        return F.mean_squared_error(self.fwd(x),y)

In [205]:
hidden_u = 20
MyModel = IrisChain(hidden_u)
MyOptimizer = optimizers.Adam()
MyOptimizer.setup(MyModel)

In [208]:
Ntrain = 10000
for i in range(Ntrain):
    x = Variable(xtrain)
    y = Variable(ytrain)
    MyModel.zerograds()
    loss = MyModel(x,y)
    loss.backward()
    MyOptimizer.update()

In [209]:
xt = Variable(xtest)
yt = MyModel.fwd(xt)

ans = yt.data # read variable/number
nrow, ncol = ans.shape
ok = 0
for i in range(nrow):
    cls = np.argmax(ans[i,:])
    if cls == yans[i]:
        ok += 1
        
print(ok, "/", nrow, "=", (ok*1.0)/nrow)

74 / 75 = 0.9866666666666667
